# Retrieving SMAP data

Data Product:
* Level 3 - daily gridded, 9km enhanced

Documentation: https://nsidc.org/sites/default/files/multi_spl4smau-v004-userguide_4.pdf

## Setup

In [ ]:
import earthaccess
import pandas
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
earthaccess.login()

## Retrieve

In [ ]:
collection = "SPL4SMGP"   
version = "008"
time_range = ("2023-01-01", "2023-01-02")

granules = earthaccess.search_data(
    short_name=collection,
    version=version,
    temporal=time_range
)

print(f"Found {len(granules)} granules")

## Read and Explore

In [ ]:
files = earthaccess.open(granules)

In [ ]:
fname = files[1]
print(f"Reading file: {fname}")
time_info = {}


# Get Meta
with h5py.File(fname, "r") as f:
    print(list(f.keys())) 
    meta = f["Metadata"]
    print("Metadata keys:", list(meta.keys()))

In [ ]:

proj_info = {}

with h5py.File(fname, "r") as f:
    
    
    dset = f["EASE2_global_projection"]
    for key, val in dset.attrs.items():
        # Convert bytes → string if needed
        if isinstance(val, bytes):
            val = val.decode()
        proj_info[key] = val

print(proj_info)

In [ ]:
with h5py.File(fname, "r") as f:
    geo = f["Geophysical_Data"]
    fields = list(geo.keys())

sm_vars = [k for k in fields if k.startswith("sm_")]
print(sm_vars)

In [ ]:
with h5py.File(fname, "r") as f:
    x = f["x"][:]          # 1D array
    y = f["y"][:]          # 1D array

    # pick one field for testing
    data = f["Geophysical_Data"]["sm_surface"][:]  

print("x shape:", x.shape)
print("y shape:", y.shape)
print("data shape:", data.shape)
print("data dtype:", data.dtype)

In [ ]:
masked_data = np.where(data == -9999.0, np.nan, data)
masked_data = np.flipud(masked_data)

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

plt.figure(figsize=(10,6))
plt.imshow(
    masked_data,
    extent=(xmin, xmax, ymin, ymax),
    origin="lower",
    interpolation="nearest"
)

plt.colorbar(label="soil_temp_layer1")
plt.title("SMAP L4 soil temperature (layer1)")
plt.xlabel("x (projection coords)")
plt.ylabel("y (projection coords)")
plt.show()